In [1]:
import pandas as pd
import requests
import time
from tqdm import tqdm

FILES_BASE_PATH = '../data/processed/'
file_path = 'api_carrer_info_cleaned.csv'


In [2]:
'''esto no estará'''

def create_df_from_csv_file(file_name):
    '''recives an csv path and returns a pandas DF'''
    file_path = FILES_BASE_PATH + file_name
    new_df = pd.read_csv(file_path)
    return new_df

In [3]:
'trameos lista raw y creamos sublistas con el codido de trabajos, segn sean high , medicum, low...'

file_path = '01_FULL_raw_table.csv'
full_raw_df = create_df_from_csv_file(file_path)


In [4]:
def make_sub_by_cat(full_rawdf, text):
    ''' makes a list from a df wich is filtered by a given value'''
    
    reduced_raw = full_raw_df[['uuid','education_level','job_code']]
    reduced_raw = reduced_raw[reduced_raw['job_code'].notna()]
    reduced_raw = reduced_raw[reduced_raw['education_level'].notna()]
    
    
    filter_edu = reduced_raw['education_level'] == text
    list_of_works = reduced_raw[filter_edu]['job_code'].to_list()
    
    
    return list(set(list_of_works))

In [5]:
def api_skill_json (list_of_jobs,text):
    '''returns a json file with the api info for each value of a given job list'''
    
    base_url= 'http://api.dataatwork.org/v1/jobs/'
    append_url = '/related_skills'

    json = []
    loop = tqdm(total=len(list_of_jobs),position = 0, leave= False)
       
    for i, job_cod in enumerate(list_of_jobs):
        loop.set_description(f'loading skills from {text.upper()} EDUCATION JOBS..'.format(i))
        loop.update(1)
        url = base_url + job_cod + append_url

        response = requests.get(url)
        job_info = response.json()
        json.append(job_info)

    loop.close()
    return json
    

In [6]:
def create_list_skills_from_json(json):
    '''from a json given returns a list of skills'''
    
    list_of_skills  = []
    
    for i in range(len(json)):
        try:
            for e in range(10):
                skill = json[i]['skills'][e]['skill_name']
                list_of_skills.append(skill)

        except:
            pass
        
    return list_of_skills
            
    
    

In [7]:
def created_sorted_dictionary (list_of_skills):
    '''from a list of skills, returns an ordered a dictionary where the key is the skill and the value the number of times wich appeas in the list'''
    base_dict = {i:list_of_skills.count(i) for i in list_of_skills}
    ordered_dict = sorted(base_dict.items(), key=lambda x: x[1], reverse=True)
    
    return ordered_dict

In [14]:
def extract_top_skills(full_raw_df,text):
    '''combines all the above functions and return a list of the top 10 skills for a given level of education'''
    
    list_jobs_by_ed = make_sub_by_cat(full_raw_df,text)
    
    json = api_skill_json(list_jobs_by_ed,text) # capar aquí con api_skill_json(list_skills[:X],text)
    
    list_of_skills = create_list_skills_from_json(json)
    
    top_10_skills = created_sorted_dictionary (list_of_skills)[:10]
    
    lista = [f'{a[0].title()} ({a[1]})' for a in top_10_skills]
    
    return lista

In [9]:
# AGREGATED FUNCTION:
def create_df_and_csv(full_raw_df,options):
    'makes a df and a csv file with all the info'
    

    dictionary = {option:extract_top_skills(full_raw_df,option) for option in options}

    df = pd.DataFrame()
    df = pd.DataFrame(dictionary).T.reset_index()

    df.columns = ['Education Level','1st Skill','2nd Skill','3th Skill','4th Skill','5th Sikill',
                  '6th Skill','7th Skill','8th Skill','9th Skill','10th Skill']
    return df

In [10]:
options = ['high','medium','low','no']


In [15]:
create_df_and_csv(full_raw_df,options)

loading skills from MEDIUM EDUCATION JOBS..:  45%|████▌     | 70/155 [01:06<01:07,  1.25it/s]

ConnectionError: ('Connection aborted.', OSError(113, 'No route to host'))